In [1]:
from test_utils import *
from swayam import Agent

### Function Calling with Agent

Let's now start to link the functionality of an LLM Agent with a locally available function call.

#### Let's see what happens without a function call

In [2]:
prompt = "What's the weather like in Boston?"

In [3]:
from swayam import Swayam
Swayam.execute(prompt)

Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
What's the weather like in Boston?
--------------------------------------------------------------------------------
Response:
I don't have real-time data access to provide current weather conditions. For the latest weather updates in Boston, I recommend checking a reliable weather website or app.
--------------------------------------------------------------------------------


ChatCompletionMessage(content="I don't have real-time data access to provide current weather conditions. For the latest weather updates in Boston, I recommend checking a reliable weather website or app.", refusal=None, role='assistant', function_call=None, tool_calls=None)

In [4]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [5]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [6]:
from swayam import Agent
agent = Agent(display=True)
function_json = agent.execute(prompt, functions=functions).function_call


Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
What's the weather like in Boston?
--------------------------------------------------------------------------------
Response:
None
--------------------------------------------------------------------------------
FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather')


In [7]:
from openai.types.chat.chat_completion_message import FunctionCall
eval(function_json.name)(**json.loads(function_json.arguments))

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'